# DuckDB and geoparquet

## Imports

In [1]:
import os
import duckdb
import folium
import pystac
import folium.plugins as folium_plugins
import geopandas as gpd
import pygeofilter
from stac_geoparquet.arrow import stac_table_to_items
# from pygeofilter.backends.sql import to_sql_where
from pygeofilter.util import IdempotentDict
from collections import OrderedDict
from pygeofilter.parsers.cql2_json import parse as json_parse

In [2]:
# Install and load DuckDB spatial extension
duckdb.install_extension("spatial")
duckdb.load_extension("spatial")

In [3]:
s2_parquet_path = "s2-stac-api.parquet"

## Query the geoparquet file

In [4]:
if not os.path.exists(s2_parquet_path):
    print("Run notebook 02-Create a geoparquet file.ipynb first")
    exit(1)

In [5]:
sql_query = f"SELECT * EXCLUDE(geometry), ST_AsWKB(geometry) as geometry FROM '{s2_parquet_path}'"

print(sql_query)

db = duckdb.query(sql_query)

SELECT * EXCLUDE(geometry), ST_AsWKB(geometry) as geometry FROM 's2-stac-api.parquet'


In [6]:
type(db)

duckdb.duckdb.DuckDBPyRelation

## Convert DuckDB result to Arrow table

In [7]:
table = db.fetch_arrow_table()

#table_head = table.slice(0, 125)  # Get first 125 rows

In [8]:
# table.to_pandas()

In [9]:
table.slice(0, 4).to_pandas()

,assets,bbox,collection,id,links,stac_extensions,stac_version,type,constellation,datetime,...,s2:reflectance_conversion_factor,s2:saturated_defective_pixel_percentage,s2:snow_ice_percentage,s2:thin_cirrus_percentage,s2:unclassified_percentage,s2:vegetation_percentage,s2:water_percentage,sat:orbit_state,sat:relative_orbit,geometry
0,"{'AOT': {'gsd': 10.0, 'href': 'https://sentine...","{'xmin': -10.25298175, 'ymin': 43.25815001, 'x...",sentinel-2-l2a,S2B_MSIL2A_20231231T114409_R123_T29TMJ_2023123...,[{'href': 'https://planetarycomputer.microsoft...,[https://stac-extensions.github.io/eo/v1.1.0/s...,1.1.0,Feature,Sentinel 2,2023-12-31 11:44:09.024000+00:00,...,1.034040,0.0,0.000000,40.464744,0.000000,0.000000,0.708269,descending,123,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x0e\x00...
1,"{'AOT': {'gsd': 10.0, 'href': 'https://sentine...","{'xmin': 0.45686052, 'ymin': 44.13799675, 'xma...",sentinel-2-l2a,S2A_MSIL2A_20231231T105441_R051_T31TCK_2023123...,[{'href': 'https://planetarycomputer.microsoft...,[https://stac-extensions.github.io/eo/v1.1.0/s...,1.1.0,Feature,Sentinel 2,2023-12-31 10:54:41.024000+00:00,...,1.034038,0.0,0.129087,13.664892,0.000000,0.000000,0.001739,descending,51,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...
2,"{'AOT': {'gsd': 10.0, 'href': 'https://sentine...","{'xmin': 0.4959286, 'ymin': 43.2382626, 'xmax'...",sentinel-2-l2a,S2A_MSIL2A_20231231T105441_R051_T31TCJ_2023123...,[{'href': 'https://planetarycomputer.microsoft...,[https://stac-extensions.github.io/eo/v1.1.0/s...,1.1.0,Feature,Sentinel 2,2023-12-31 10:54:41.024000+00:00,...,1.034038,0.0,0.000010,18.065368,0.000086,0.000000,0.010481,descending,51,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...
3,"{'AOT': {'gsd': 10.0, 'href': 'https://sentine...","{'xmin': 0.53321795, 'ymin': 42.33836183, 'xma...",sentinel-2-l2a,S2A_MSIL2A_20231231T105441_R051_T31TCH_2023123...,[{'href': 'https://planetarycomputer.microsoft...,[https://stac-extensions.github.io/eo/v1.1.0/s...,1.1.0,Feature,Sentinel 2,2023-12-31 10:54:41.024000+00:00,...,1.034038,0.0,0.055783,4.123205,0.047372,0.522049,0.014681,descending,51,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...


In [10]:
table_head = table.slice(0, 125)  # Get first 125 rows
# for item in stac_table_to_items(table_head):
#     print(item)

In [11]:
table_head.to_pandas()

,assets,bbox,collection,id,links,stac_extensions,stac_version,type,constellation,datetime,...,s2:reflectance_conversion_factor,s2:saturated_defective_pixel_percentage,s2:snow_ice_percentage,s2:thin_cirrus_percentage,s2:unclassified_percentage,s2:vegetation_percentage,s2:water_percentage,sat:orbit_state,sat:relative_orbit,geometry
0,"{'AOT': {'gsd': 10.0, 'href': 'https://sentine...","{'xmin': -10.25298175, 'ymin': 43.25815001, 'x...",sentinel-2-l2a,S2B_MSIL2A_20231231T114409_R123_T29TMJ_2023123...,[{'href': 'https://planetarycomputer.microsoft...,[https://stac-extensions.github.io/eo/v1.1.0/s...,1.1.0,Feature,Sentinel 2,2023-12-31 11:44:09.024000+00:00,...,1.034040,0.0,0.000000,40.464744,0.000000,0.000000,0.708269,descending,123,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x0e\x00...
1,"{'AOT': {'gsd': 10.0, 'href': 'https://sentine...","{'xmin': 0.45686052, 'ymin': 44.13799675, 'xma...",sentinel-2-l2a,S2A_MSIL2A_20231231T105441_R051_T31TCK_2023123...,[{'href': 'https://planetarycomputer.microsoft...,[https://stac-extensions.github.io/eo/v1.1.0/s...,1.1.0,Feature,Sentinel 2,2023-12-31 10:54:41.024000+00:00,...,1.034038,0.0,0.129087,13.664892,0.000000,0.000000,0.001739,descending,51,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...
2,"{'AOT': {'gsd': 10.0, 'href': 'https://sentine...","{'xmin': 0.4959286, 'ymin': 43.2382626, 'xmax'...",sentinel-2-l2a,S2A_MSIL2A_20231231T105441_R051_T31TCJ_2023123...,[{'href': 'https://planetarycomputer.microsoft...,[https://stac-extensions.github.io/eo/v1.1.0/s...,1.1.0,Feature,Sentinel 2,2023-12-31 10:54:41.024000+00:00,...,1.034038,0.0,0.000010,18.065368,0.000086,0.000000,0.010481,descending,51,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...
3,"{'AOT': {'gsd': 10.0, 'href': 'https://sentine...","{'xmin': 0.53321795, 'ymin': 42.33836183, 'xma...",sentinel-2-l2a,S2A_MSIL2A_20231231T105441_R051_T31TCH_2023123...,[{'href': 'https://planetarycomputer.microsoft...,[https://stac-extensions.github.io/eo/v1.1.0/s...,1.1.0,Feature,Sentinel 2,2023-12-31 10:54:41.024000+00:00,...,1.034038,0.0,0.055783,4.123205,0.047372,0.522049,0.014681,descending,51,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...
4,"{'AOT': {'gsd': 10.0, 'href': 'https://sentine...","{'xmin': 0.5688037, 'ymin': 41.4388363, 'xmax'...",sentinel-2-l2a,S2A_MSIL2A_20231231T105441_R051_T31TCG_2023123...,[{'href': 'https://planetarycomputer.microsoft...,[https://stac-extensions.github.io/eo/v1.1.0/s...,1.1.0,Feature,Sentinel 2,2023-12-31 10:54:41.024000+00:00,...,1.034038,0.0,0.000000,4.116742,0.269664,0.144867,0.097903,descending,51,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,"{'AOT': {'gsd': 10.0, 'href': 'https://sentine...","{'xmin': -4.67851734, 'ymin': 39.64771538, 'xm...",sentinel-2-l2a,S2B_MSIL2A_20231229T110359_R094_T30TUK_2023122...,[{'href': 'https://planetarycomputer.microsoft...,[https://stac-extensions.github.io/eo/v1.1.0/s...,1.1.0,Feature,Sentinel 2,2023-12-29 11:03:59.024000+00:00,...,1.033865,0.0,0.000046,0.587879,0.000000,0.000000,0.000865,descending,94,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x0c\x00...
121,"{'AOT': {'gsd': 10.0, 'href': 'https://sentine...","{'xmin': -1.84912778, 'ymin': 38.74919438, 'xm...",sentinel-2-l2a,S2B_MSIL2A_20231229T110359_R094_T30SXJ_2023122...,[{'href': 'https://planetarycomputer.microsoft...,[https://stac-extensions.github.io/eo/v1.1.0/s...,1.1.0,Feature,Sentinel 2,2023-12-29 11:03:59.024000+00:00,...,1.033865,0.0,0.000000,0.000008,0.000000,0.000000,0.000000,descending,94,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x0c\x00...
122,"{'AOT': {'gsd': 10.0, 'href': 'https://sentine...","{'xmin': -1.8632824, 'ymin': 37.8524187, 'xmax...",sentinel-2-l2a,S2B_MSIL2A_20231229T110359_R094_T30SXH_2023122...,[{'href': 'https://planetarycomputer.microsoft...,[https://stac-extensions.github.io/eo/v1.1.0/s...,1.1.0,Feature,Sentinel 2,2023-12-29 11:03:59.024000+00:00,...,1.033865,0.0,0.000000,0.000000,0.000000,0.000000,0.0

## Map


In [12]:
all_items = [pystac.Item.from_dict(item) for item in stac_table_to_items(table)]
all_items

[<Item id=S2B_MSIL2A_20231231T114409_R123_T29TMJ_20231231T150806>,
 <Item id=S2A_MSIL2A_20231231T105441_R051_T31TCK_20231231T162343>,
 <Item id=S2A_MSIL2A_20231231T105441_R051_T31TCJ_20231231T162355>,
 <Item id=S2A_MSIL2A_20231231T105441_R051_T31TCH_20231231T162350>,
 <Item id=S2A_MSIL2A_20231231T105441_R051_T31TCG_20231231T162354>,
 <Item id=S2A_MSIL2A_20231231T105441_R051_T31TCF_20231231T162151>,
 <Item id=S2A_MSIL2A_20231231T105441_R051_T31TCE_20231231T162147>,
 <Item id=S2A_MSIL2A_20231231T105441_R051_T31TBG_20231231T162352>,
 <Item id=S2A_MSIL2A_20231231T105441_R051_T31TBF_20231231T162153>,
 <Item id=S2A_MSIL2A_20231231T105441_R051_T31TBE_20231231T162152>,
 <Item id=S2A_MSIL2A_20231231T105441_R051_T31SCD_20231231T162143>,
 <Item id=S2A_MSIL2A_20231231T105441_R051_T31SCC_20231231T162140>,
 <Item id=S2A_MSIL2A_20231231T105441_R051_T31SCB_20231231T162138>,
 <Item id=S2A_MSIL2A_20231231T105441_R051_T31SBV_20231231T162142>,
 <Item id=S2A_MSIL2A_20231231T105441_R051_T31SBD_20231231T1621

In [13]:
items = [pystac.Item.from_dict(item) for item in stac_table_to_items(table_head)]
items[0]

<Item id=S2B_MSIL2A_20231231T114409_R123_T29TMJ_20231231T150806>

In [15]:
# items = [pystac.Item.from_dict(item) for item in stac_table_to_items(table_head)]
print(len(all_items))
map = folium.Map(zoom_start=6)
layer_control = folium.LayerControl(position="topright", collapsed=True)
fullscreen = folium_plugins.Fullscreen()
style = {"fillColor": "#00000000", "color": "#0000ff", "weight": 1}

footprints = folium.GeoJson(
    gpd.GeoDataFrame.from_features(all_items).to_json(),
    name="Stac Item footprints",
    style_function=lambda x: style,
    control=True,
)

footprints.add_to(map)
layer_control.add_to(map)
fullscreen.add_to(map)
map.fit_bounds(map.get_bounds())
map

1000


### Geofilter

In [19]:
from pygeofilter.parsers.cql2_json import parse as json_parse
from pygeofilter.util import IdempotentDict
from collections import OrderedDict
from pygeofilter_duckdb import to_sql_where

cql2_filter = {
    "op": "and",
    "args": [
        {
            "op": "between",
            "args": [
                {"property": "datetime"},
                ["2023-12-28T00:00:00Z", "2023-12-28T23:59:59Z"],
            ],
        },
        {"op": "between", "args": [{"property": "eo:cloud_cover"}, [90, 100]]},
        {
            "op": "s_intersects",
            "args": [
                {"property": "geometry"},
                {
                    "type": "Polygon",
                    "coordinates": [
                        [
                            [-9.51346013858793, 38.95450355515311],
                            [-9.51346013858793, 38.22500810801125],
                            [-8.359265560322228, 38.22500810801125],
                            [-8.359265560322228, 38.95450355515311],
                            [-9.51346013858793, 38.95450355515311],
                        ]
                    ],
                },
            ],
        },
    ],
}

# Convert to SQL WHERE clause
sql_where = to_sql_where(json_parse(cql2_filter), IdempotentDict())
print(sql_where)

((("datetime" BETWEEN '2023-12-28T00:00:00Z' AND '2023-12-28T23:59:59Z') AND ("eo:cloud_cover" BETWEEN 90 AND 100)) AND ST_Intersects("geometry",ST_GeomFromHEXEWKB('01030000000100000005000000D0114E3FE40623C030A6282C2D7A4340D0114E3FE40623C0209FD010CD1C43408053D0A7F1B720C0209FD010CD1C43408053D0A7F1B720C030A6282C2D7A4340D0114E3FE40623C030A6282C2D7A4340')))


In [20]:
sql_query = f"SELECT * EXCLUDE(geometry), ST_AsWKB(geometry) as geometry FROM '{s2_parquet_path}' WHERE {sql_where}"
db_filtered = duckdb.query(sql_query)

In [23]:
subset_table = db_filtered.fetch_arrow_table()
subset_table[0]

[
  -- is_valid: all not null
  -- child 0 type: struct<gsd: double, href: string, proj:bbox: list<l: double>, proj:shape: list<l: int64>, proj:transform: list<l: double>, roles: list<l: string>, title: string, type: string>
    -- is_valid: all not null
    -- child 0 type: double
      [
        10,
        10,
        10,
        10
      ]
    -- child 1 type: string
      [
        "https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/29/S/ND/2023/12/28/S2B_MSIL2A_20231228T113409_N0510_R080_T29SND_20231228T144647.SAFE/GRANULE/L2A_T29SND_A035570_20231228T113409/IMG_DATA/R10m/T29SND_20231228T113409_AOT_10m.tif",
        "https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/29/S/NC/2023/12/28/S2B_MSIL2A_20231228T113409_N0510_R080_T29SNC_20231228T142841.SAFE/GRANULE/L2A_T29SNC_A035570_20231228T113409/IMG_DATA/R10m/T29SNC_20231228T113409_AOT_10m.tif",
        "https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/29/S/MD/2023/12/28/S2B_MSIL2A_20231228T113409_N0510_R080_T2

In [25]:
items = [pystac.Item.from_dict(item) for item in stac_table_to_items(subset_table)]
print(len(items))
display(items[0])

4


<Item id=S2B_MSIL2A_20231228T113409_R080_T29SND_20231228T144647>

In [27]:
map = folium.Map()
layer_control = folium.LayerControl(position="topright", collapsed=True)
fullscreen = folium_plugins.Fullscreen()
style = {"fillColor": "#00000000", "color": "#0000ff", "weight": 1}

footprints = folium.GeoJson(
    gpd.GeoDataFrame.from_features(items).to_json(),
    name="Stac Item footprints",
    style_function=lambda x: style,
    control=True,
)

footprints.add_to(map)
layer_control.add_to(map)
fullscreen.add_to(map)
map.fit_bounds(map.get_bounds())
map